In [6]:
import json
import numpy as np

In [4]:
with open("dataset/train-v2.0.json", 'r') as handle:
    jdata = json.load(handle)
    data = jdata['data']

In [5]:
contexts = []
questions = []
unanswerable = []
answerable = []
for i in range(len(data)):
    section = data[i]['paragraphs']
    for sec in section:
        context = sec['context']
        contexts.append(context)
        qas = sec['qas']
        for j in range(len(qas)):
            question = qas[j]['question']
            questions.append(question)
            label = qas[j]['is_impossible']
            if label:
                unanswerable.append((len(contexts)-1, len(questions)-1))
            else:
                answerable.append((len(contexts)-1, len(questions)-1))

In [89]:
i = np.random.choice(range(len(answerable)))
c_i, q_i = answerable[i]
print(contexts[c_i])
print("##"*10)
print(questions[q_i])

From 1989 through 1996, the total area of the US was listed as 9,372,610 km2 (3,618,780 sq mi) (land + inland water only). The listed total area changed to 9,629,091 km2 (3,717,813 sq mi) in 1997 (Great Lakes area and coastal waters added), to 9,631,418 km2 (3,718,711 sq mi) in 2004, to 9,631,420 km2 (3,718,710 sq mi) in 2006, and to 9,826,630 km2 (3,794,080 sq mi) in 2007 (territorial waters added). Currently, the CIA World Factbook gives 9,826,675 km2 (3,794,100 sq mi), the United Nations Statistics Division gives 9,629,091 km2 (3,717,813 sq mi), and the Encyclopædia Britannica gives 9,522,055 km2 (3,676,486 sq mi)(Great Lakes area included but not coastal waters). These source consider only the 50 states and the Federal District, and exclude overseas territories.
####################
According to the Encyclopedia Britannica, what is the total area of the US in miles?


# Alignment test

In [66]:
from collections import Counter

In [14]:
import torch
from pytorch_pretrained_bert import BertTokenizer, BertModel

Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.


In [37]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')


def test(sent1, sent2):
    # Tokenized input
    sent1_tokens =  tokenizer.tokenize(sent1)
    sent2_tokens =  tokenizer.tokenize(sent2)
    tokenized_text = tokenizer.tokenize(sent1 + " " + sent2)
    print(len(tokenized_text))

    # Convert token to vocabulary indices
    indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
    # Define sentence A and B indices associated to 1st and 2nd sentences (see paper)
    segments_ids = [0]*len(sent1_tokens) + [1]*len(sent2_tokens)

    # Convert inputs to PyTorch tensors
    tokens_tensor = torch.tensor([indexed_tokens])
    segments_tensors = torch.tensor([segments_ids])

    model = BertModel.from_pretrained('bert-base-uncased')
    model.eval()

    # Predict all tokens
    output = model(tokens_tensor,  segments_tensors)
    return output,tokenized_text






In [90]:
output, tokenized_text = test(contexts[c_i], questions[q_i])

287


In [91]:
print(contexts[c_i])
print(questions[q_i])
print (list(zip(tokenized_text, range(len(tokenized_text)))))

From 1989 through 1996, the total area of the US was listed as 9,372,610 km2 (3,618,780 sq mi) (land + inland water only). The listed total area changed to 9,629,091 km2 (3,717,813 sq mi) in 1997 (Great Lakes area and coastal waters added), to 9,631,418 km2 (3,718,711 sq mi) in 2004, to 9,631,420 km2 (3,718,710 sq mi) in 2006, and to 9,826,630 km2 (3,794,080 sq mi) in 2007 (territorial waters added). Currently, the CIA World Factbook gives 9,826,675 km2 (3,794,100 sq mi), the United Nations Statistics Division gives 9,629,091 km2 (3,717,813 sq mi), and the Encyclopædia Britannica gives 9,522,055 km2 (3,676,486 sq mi)(Great Lakes area included but not coastal waters). These source consider only the 50 states and the Federal District, and exclude overseas territories.
According to the Encyclopedia Britannica, what is the total area of the US in miles?
[('from', 0), ('1989', 1), ('through', 2), ('1996', 3), (',', 4), ('the', 5), ('total', 6), ('area', 7), ('of', 8), ('the', 9), ('us', 10)

In [96]:
key_index = 266
min_dist = 99999
min_index = -1
close_terms = Counter()
for word, index in zip(tokenized_text, range(len(tokenized_text))):
    if index == key_index :#or tokenized_text[index] == tokenized_text[key_index]:
        continue
    dist = np.linalg.norm(output[0][-1][0,key_index,:].data.numpy()
                          -output[0][-1][0,index,:].data.numpy())
    close_terms[(tokenized_text[index], index)] = dist
    if dist < min_dist:
        min_dist = dist
        min_index = index
print(list(reversed(close_terms.most_common()))[:5])  #(tokenized_text[min_index], min_index)

[(('overseas', 265), 7.748758), (('district', 261), 9.652206), (('and', 258), 10.993108), (('federal', 260), 11.238921), (('waters', 248), 11.343638)]


In [23]:
all_encoder_layers, _ = output
all_encoder_layers = all_encoder_layers

for b, example_index in enumerate(example_indices):
    feature = features[example_index.item()]
    unique_id = int(feature.unique_id)
    # feature = unique_id_to_feature[unique_id]
    output_json = collections.OrderedDict()
    output_json["linex_index"] = unique_id
    all_out_features = []
    for (i, token) in enumerate(feature.tokens):
        all_layers = []
        for (j, layer_index) in enumerate(layer_indexes):
            layer_output = all_encoder_layers[int(layer_index)].detach().cpu().numpy()
            layer_output = layer_output[b]
            layers = collections.OrderedDict()
            layers["index"] = layer_index
            layers["values"] = [
                round(x.item(), 6) for x in layer_output[i]
            ]
            all_layers.append(layers)
        out_features = collections.OrderedDict()
        out_features["token"] = token
        out_features["layers"] = all_layers
        all_out_features.append(out_features)
    output_json["features"] = all_out_features

12